In [ ]:
import pandas as pd
# import datetime

In [ ]:
FILE_NAME = 'test'
OUTPUT_DATA_FILE = f'{FILE_NAME}_data.parquet'
OUTPUT_METADATA_FILE = f'{FILE_NAME}_meta.parquet'

In [ ]:
data = pd.read_parquet(f'{OUTPUT_DATA_FILE}')
metadata = pd.read_parquet(f'{OUTPUT_METADATA_FILE}')

In [ ]:
data

In [ ]:
metadata

In [ ]:
data['author'].value_counts()[:10]

In [ ]:
data.info()

In [ ]:
metadata.info()